In [95]:
# Stuff Documents 체인을 사용하여 완전한 RAG 파이프라인을 구현하세요.
# 체인을 수동으로 구현해야 합니다.
# 체인에 ConversationBufferMemory를 부여합니다.
# 이 문서를 사용하여 RAG를 수행하세요: https://gist.github.com/serranoarevalo/5acf755c2b8d83f1707ef266b82ea223
# 체인에 다음 질문을 합니다:
# Aaronson 은 유죄인가요?
# 그가 테이블에 어떤 메시지를 썼나요?
# Julia 는 누구인가요?
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(
    temperature=0.1,
    model="gpt-4o-mini-2024-07-18",
)

memory = ConversationBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True,
)

embedding_folder = 'embeddings'
file_folder = 'files'
file_name = 'document.txt'

cache_dir = LocalFileStore(f'./.cache/{embedding_folder}/{file_name}')

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader(f'./{file_folder}/{file_name}')

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cache_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cache_embeddings)

retriever = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 문서 내 정보를 탐색하는데 특화된 ai입니다. 주어지는 컨텍스트만을 사용하여 답변하되, 모르는건 모른다고 하고 지어내지 마세요.\n{context}"),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{question}")
])

def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = {"context": retriever, "question": RunnablePassthrough(), "history": load_memory,} | prompt | llm

def invoke_chain(question):
    result = chain.invoke(question)
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)

In [96]:
invoke_chain("Aaronson 은 guilty인가요?")

content='문서의 내용에 따르면, Winston은 Aaronson, Jones, 그리고 Rutherford가 그들이 기소된 범죄에 대해 유죄라고 믿고 있었지만, 그는 그들의 무죄를 증명하는 사진을 본 적이 없다고 언급합니다. 그는 그 사진이 존재하지 않았고, 자신이 그것을 발명했다고 생각합니다. 따라서, Winston은 Aaronson이 유죄라고 믿고 있었지만, 이는 잘못된 기억의 산물이라고 결론짓고 있습니다.'


In [97]:
invoke_chain("그가 테이블에 어떤 메시지를 썼나요?")

content='Winston은 테이블에 다음과 같은 메시지를 썼습니다:\n\n1. "FREEDOM IS SLAVERY"\n2. "TWO AND TWO MAKE FIVE"\n3. "GOD IS POWER" \n\n이 메시지들은 그가 자신의 생각을 기록하는 과정에서 나온 것입니다.'


In [98]:
invoke_chain("Julia 는 누구인가요?")

content='Julia는 Winston의 사랑하는 사람으로, 그와 함께 반당파적인 관계를 맺고 있는 인물입니다. 그녀는 Winston과의 관계에서 자유롭고 개인적인 감정을 표현하며, 그와 함께 당의 억압에 저항하려고 합니다. 문서에서는 Winston이 Julia에 대한 강한 감정을 느끼고 있으며, 그녀의 안전과 행복을 걱정하는 모습이 나타납니다.'


In [99]:
memory

ConversationBufferMemory(chat_memory=ChatMessageHistory(messages=[HumanMessage(content='Aaronson 은 guilty인가요?'), AIMessage(content='문서의 내용에 따르면, Winston은 Aaronson, Jones, 그리고 Rutherford가 그들이 기소된 범죄에 대해 유죄라고 믿고 있었지만, 그는 그들의 무죄를 증명하는 사진을 본 적이 없다고 언급합니다. 그는 그 사진이 존재하지 않았고, 자신이 그것을 발명했다고 생각합니다. 따라서, Winston은 Aaronson이 유죄라고 믿고 있었지만, 이는 잘못된 기억의 산물이라고 결론짓고 있습니다.'), HumanMessage(content='그가 테이블에 어떤 메시지를 썼나요?'), AIMessage(content='Winston은 테이블에 다음과 같은 메시지를 썼습니다:\n\n1. "FREEDOM IS SLAVERY"\n2. "TWO AND TWO MAKE FIVE"\n3. "GOD IS POWER" \n\n이 메시지들은 그가 자신의 생각을 기록하는 과정에서 나온 것입니다.'), HumanMessage(content='Julia 는 누구인가요?'), AIMessage(content='Julia는 Winston의 사랑하는 사람으로, 그와 함께 반당파적인 관계를 맺고 있는 인물입니다. 그녀는 Winston과의 관계에서 자유롭고 개인적인 감정을 표현하며, 그와 함께 당의 억압에 저항하려고 합니다. 문서에서는 Winston이 Julia에 대한 강한 감정을 느끼고 있으며, 그녀의 안전과 행복을 걱정하는 모습이 나타납니다.')]), return_messages=True)